Long Short Term Memory (LSTM)

Exe. 1

In [ ]:
# Upload the price data set, data.csv using pandas. Check the uploaded dataset and its different columns
import pandas as pd
from google.colab import files

uploaded = files.upload()
prices_dataset = pd.read_csv('data.csv')

prices_dataset.head()

Exe. 2

In [ ]:
# Regarding the uploaded dataset, the ’symbol’ column contains a shortbname of each company.
# For instance YHOO represent Yahoo. Get a subsetnof dataset, contaning YHOO symbol. name this subset of data as yahoo. 
# Get only the high column values (6th column) and save them in a variable namely yahoo stock prices, and demonstrate high values in a graph.
yahoo = prices_dataset[prices_dataset['symbol']=='YHOO']
print(yahoo.head())
yahoo_stock_prices = yahoo.values[:,-2].astype('float32')
print(yahoo_stock_prices.shape)
yahoo_stock_prices = yahoo_stock_prices.reshape(yahoo_stock_prices.shape[0], 1)
print(yahoo_stock_prices)

Exe. 3

In [ ]:
# Now divide the yahoo stock prices into train and test variables.
# For instance 80% for train and 20% for the test set. 
# Do NOT shuffle the data set because it is a sequential dataset and the its order is important.
train_size = int(len(yahoo_stock_prices) * 0.80)
test_size = len(yahoo_stock_prices) - train_size
train, test = yahoo_stock_prices[0:train_size], yahoo_stock_prices[train_size:len(yahoo_stock_prices)]
print(len(train), len(test))

Exe. 4

In [ ]:
import numpy as np

def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

Exe. 5

In [ ]:
# Define a neural network model as such : LSTM, Droupout, LSTM, Droupout, Dense, Activation.

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation

model = Sequential()
model.add(LSTM(input_dim = 1, units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation = 'linear'))

model.summary()

#Compile the model using a mean square error and a rmsprop optimisation.
model.compile(loss='mse', optimizer='rmsprop')

Exe. 6

In [ ]:
# Now fit the model on tainX and trainY datasets while defining your batch size, epochs and validation split.
model.fit(trainX, trainY, batch_size=1, epochs=5, validation_split=0.05)

Exe. 7

In [ ]:
# Predict the textX with the trained LSTM model and demonstrate the exact and predicted value in a graph. 
# What is your observation?
predicted = model.predict(testX)
print(predicted.shape)
print(testY.shape)

import matplotlib.pyplot as plt
plt.plot(testY)
plt.plot(predicted)
plt.show()

Exe. 8

In [ ]:
# Train another model on a different company from the original dataset? How is your prediction method?

BMY = prices_dataset[prices_dataset['symbol']=='BMY']
print(BMY.head())
BMY_stock_prices = BMY.values[:,-2].astype('float32')
print(BMY_stock_prices.shape)
BMY_stock_prices = BMY_stock_prices.reshape(BMY_stock_prices.shape[0], 1)
print(BMY_stock_prices)

train_size = int(len(BMY_stock_prices) * 0.80)
test_size = len(BMY_stock_prices) - train_size
train, test = BMY_stock_prices[0:train_size], BMY_stock_prices[train_size:len(BMY_stock_prices)]
print(len(train), len(test))

trainX, trainY = create_dataset(train, 1)
testX, testY = create_dataset(test, 1)

trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

model = Sequential()
model.add(LSTM(input_dim = 1, units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(100, return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(units = 1, activation = 'linear'))

model.summary()

model.compile(loss='mse', optimizer='rmsprop')

model.fit(trainX, trainY, batch_size=1, epochs=20, validation_split=0.05)

predicted = model.predict(testX)
print(predicted.shape)
print(testY.shape)

plt.plot(testY)
plt.plot(predicted)
plt.show()

Gated Recurrent unit (GRU)

Exe. 9

In [ ]:
from google.colab import files
uploaded = files.upload()

# Reading the text file into a string 
with open('poems.txt', 'r') as file: 
	text = file.read() 

# A preview of the text file	 
print(text)

Exe. 10

In [ ]:
#storing all the unique characters present in the text
vocabulary = sorted(list(set(text)))

#Creating dictionaries to map each character to an index
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

# A preview of the dictionaries
print(vocabulary)
print(char_to_indices)
print(indices_to_char)

Exe. 11

In [ ]:
max_length = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])

print('Number of sequences:', len(sentences))

# A preview of the sentences and next_chars
print(sentences[0])
print(next_chars[0])

Exe. 12

In [ ]:
import numpy as np

X = np.zeros((len(sentences), max_length , len(vocabulary)), dtype = bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

Exe. 13

In [ ]:
# model you network as GRU (128), Dense(len(vocabulary))

from keras.models import Sequential
from keras.layers import Dense, GRU

model = Sequential()
model.add(GRU(128, input_shape=(max_length, len(vocabulary))))
model.add(Dense(len(vocabulary), activation='softmax'))

model.summary()

# Finally compile the model with categorical cross entropy and a RMSprop optimiser
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

Exe. 14

In [ ]:
# fit the trained model on X and Y datasets by defining a batch size and epochs values. 
model.fit(X, y, batch_size=128, epochs=100)

# Check loss and accuracy of your model.
score = model.evaluate(X, y, batch_size=128)
print(score)

Exe. 15

In [ ]:
# Generate a text similar to the trained data
# chose part of the original text randomly and separate a 100 size text from it.
start_index = np.random.randint(0, len(text) - max_length - 1)
generated_text = text[start_index: start_index + max_length]
print('--- Generating with seed: "' + generated_text + '"')

# Generate 100 characters
for i in range(100):
    sampled = np.zeros((1, max_length, len(vocabulary)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_to_indices[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = np.argmax(preds)
    next_char = indices_to_char[next_index]

    generated_text += next_char
    generated_text = generated_text[1:]

print(generated_text)

Exe. 16

In [ ]:
# Test similar text generator method for another text data source as Sherlock Holmes.

from google.colab import files
uploaded = files.upload()

# Reading the text file into a string
with open('sherlock.txt', 'r') as file:
    text = file.read()

# A preview of the text file
print(text)

#storing all the unique characters present in the text
vocabulary = sorted(list(set(text)))

#Creating dictionaries to map each character to an index
char_to_indices = dict((c, i) for i, c in enumerate(vocabulary))
indices_to_char = dict((i, c) for i, c in enumerate(vocabulary))

# A preview of the dictionaries
print(vocabulary)
print(char_to_indices)
print(indices_to_char)

max_length = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - max_length, step):
    sentences.append(text[i: i + max_length])
    next_chars.append(text[i + max_length])

print('Number of sequences:', len(sentences))

# A preview of the sentences and next_chars
print(sentences[0])
print(next_chars[0])

import numpy as np

X = np.zeros((len(sentences), max_length , len(vocabulary)), dtype = bool)
y = np.zeros((len(sentences), len(vocabulary)), dtype = bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_indices[char]] = 1
    y[i, char_to_indices[next_chars[i]]] = 1

# model you network as GRU (128), Dense(len(vocabulary))

from keras.models import Sequential
from keras.layers import Dense, GRU

model = Sequential()
model.add(GRU(128, input_shape=(max_length, len(vocabulary))))
model.add(Dense(len(vocabulary), activation='softmax'))

model.summary()

# Finally compile the model with categorical cross entropy and a RMSprop optimiser
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# fit the trained model on X and Y datasets by defining a batch size and epochs values.
model.fit(X, y, batch_size=128, epochs=100)

# Check loss and accuracy of your model.
score = model.evaluate(X, y, batch_size=128)
print(score)

# Generate a text similar to the trained data
# chose part of the original text randomly and separate a 100 size text from it.
start_index = np.random.randint(0, len(text) - max_length - 1)
generated_text = text[start_index: start_index + max_length]
print('--- Generating with seed: "' + generated_text + '"')

# Generate 1000 characters
for i in range(1000):
    sampled = np.zeros((1, max_length, len(vocabulary)))
    for t, char in enumerate(generated_text):
        sampled[0, t, char_to_indices[char]] = 1.

    preds = model.predict(sampled, verbose=0)[0]
    next_index = np.argmax(preds)
    next_char = indices_to_char[next_index]

    generated_text += next_char
    generated_text = generated_text[1:]

print(generated_text)